# Diabetes Risk Prediction (with Explainable AI)

This notebook walks you step-by-step through:
1. Loading the PIMA Indians Diabetes dataset
2. Cleaning & imputing impossible zeros
3. Feature engineering (BMI category, Glucose/BMI ratio)
4. Train/test split with stratification
5. Pipelines with SMOTE and models (LR, RF, XGB)
6. Cross-validation
7. Explainability with SHAP
8. Saving and using the pipeline


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import shap
import joblib

sns.set(context="notebook", style="whitegrid")
np.set_printoptions(suppress=True)


## Load data & sanity checks
Place your CSV at `../data/diabetes.csv`.


In [ ]:
df = pd.read_csv("../data/diabetes.csv")
print("Shape:", df.shape)
display(df.head())
df.info()
print("\nOutcome value counts:\n", df["Outcome"].value_counts())


## Treat impossible zeros as missing (NaN)
We will replace zeros in specific columns with NaN so we can impute them later.

In [ ]:
df = df.copy()
zero_invalid_cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for c in zero_invalid_cols:
    df[c] = df[c].replace(0, np.nan)
df.isna().sum()


## Quick EDA
We plot class balance, histograms, and a correlation heatmap.

In [ ]:
sns.countplot(x="Outcome", data=df)
plt.title("Outcome (0=No, 1=Yes)")
plt.show()

df.hist(figsize=(12,8))
plt.tight_layout(); plt.show()

plt.figure(figsize=(10,8))
sns.heatmap(df.corr(numeric_only=True), annot=False, cmap="coolwarm")
plt.title("Correlation heatmap")
plt.show()


## Feature engineering
- BMI Category using WHO cutoffs
- Glucose/BMI ratio

In [ ]:
def bmi_category(bmi):
    if pd.isna(bmi):
        return np.nan
    if bmi < 18.5:
        return "Underweight"
    elif bmi < 25:
        return "Normal"
    elif bmi < 30:
        return "Overweight"
    else:
        return "Obese"

df["BMI_Category"] = df["BMI"].apply(bmi_category)
df["Glucose_BMI_Ratio"] = df["Glucose"] / df["BMI"]
df.head()


## Train-test split

In [ ]:
X = df.drop(columns=["Outcome"])
y = df["Outcome"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train.shape, X_test.shape


## Pipelines with preprocessing + SMOTE + model
We'll build reusable pipelines for Logistic Regression, Random Forest, and XGBoost.

In [ ]:
cat_cols = ["BMI_Category"]
num_cols = [c for c in X.columns if c not in cat_cols]

numeric_prep = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_prep = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_prep, num_cols),
    ("cat", categorical_prep, cat_cols)
])

def evaluate(name, y_true, y_pred, y_prob):
    print(f"\n=== {name} ===")
    print("Accuracy :", round(accuracy_score(y_true, y_pred), 4))
    print("Precision:", round(precision_score(y_true, y_pred), 4))
    print("Recall   :", round(recall_score(y_true, y_pred), 4))
    print("F1-score :", round(f1_score(y_true, y_pred), 4))
    print("ROC-AUC  :", round(roc_auc_score(y_true, y_prob), 4))
    print("\nClassification report:\n", classification_report(y_true, y_pred))

    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.show()

    RocCurveDisplay.from_predictions(y_true, y_prob)
    plt.title(f"{name} - ROC Curve"); plt.show()

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def make_lr_pipeline():
    return ImbPipeline(steps=[
        ("prep", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("model", LogisticRegression(max_iter=1000))
    ])

def make_rf_pipeline():
    return ImbPipeline(steps=[
        ("prep", preprocessor),\
        ("smote", SMOTE(random_state=42)),\
        ("model", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
    ])

def make_xgb_pipeline():
    return ImbPipeline(steps=[
        ("prep", preprocessor),
        ("smote", SMOTE(random_state=42)),
        ("model", XGBClassifier(
            n_estimators=400,
            max_depth=5,
            learning_rate=0.08,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            eval_metric="logloss",
            random_state=42,
            n_jobs=-1
        ))
    ])


## Train & evaluate models

In [ ]:
lr_pipe = make_lr_pipeline().fit(X_train, y_train)
y_pred_lr = lr_pipe.predict(X_test)
y_prob_lr = lr_pipe.predict_proba(X_test)[:, 1]
evaluate("Logistic Regression", y_test, y_pred_lr, y_prob_lr)

rf_pipe = make_rf_pipeline().fit(X_train, y_train)
y_pred_rf = rf_pipe.predict(X_test)
y_prob_rf = rf_pipe.predict_proba(X_test)[:, 1]
evaluate("Random Forest", y_test, y_pred_rf, y_prob_rf)

xgb_pipe = make_xgb_pipeline().fit(X_train, y_train)
y_pred_xgb = xgb_pipe.predict(X_test)
y_prob_xgb = xgb_pipe.predict_proba(X_test)[:, 1]
evaluate("XGBoost", y_test, y_pred_xgb, y_prob_xgb)


## Cross-validation (XGBoost)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = cross_val_score(make_xgb_pipeline(), X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
print("CV ROC-AUC (XGB):", np.round(auc_scores, 4), "Mean:", round(auc_scores.mean(), 4))


## Explainability with SHAP (XGBoost)

In [ ]:
# Use the fitted XGB pipeline
prep = xgb_pipe.named_steps["prep"]
model = xgb_pipe.named_steps["model"]

# Transform training data to model space
X_train_proc = prep.transform(X_train)

try:
    num_features = preprocessor.named_transformers_["num"].get_feature_names_out([*num_cols])
except Exception:
    num_features = np.array(num_cols)

ohe = preprocessor.named_transformers_["cat"].named_steps["ohe"]
cat_features = ohe.get_feature_names_out(["BMI_Category"])  # aligned with ColumnTransformer
feature_names = np.r_[num_features, cat_features]

X_sample = X_train_proc if X_train_proc.shape[0] <= 200 else X_train_proc[:200]
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_sample)
shap.summary_plot(shap_values, X_sample, feature_names=feature_names, plot_type="bar")


## Save pipeline & predict on new data

In [ ]:
import joblib
joblib.dump(xgb_pipe, "../diabetes_risk_pipeline.joblib")
print("Saved model -> ../diabetes_risk_pipeline.joblib")

new_patient = {
    "Pregnancies": 2,
    "Glucose": 145,
    "BloodPressure": 80,
    "SkinThickness": 25,
    "Insulin": 100,
    "BMI": 31.2,
    "DiabetesPedigreeFunction": 0.35,
    "Age": 45,
    "BMI_Category": "Obese",
    "Glucose_BMI_Ratio": 145/31.2
}
new_df = pd.DataFrame([new_patient])
loaded = joblib.load("../diabetes_risk_pipeline.joblib")
prob = loaded.predict_proba(new_df)[:, 1][0]
pred = loaded.predict(new_df)[0]
print(f"Probability of diabetes: {prob:.3f} -> Predicted class: {pred}")
